### Setup

In [1]:
using Dragoon

In [2]:
#number of discs in the booster
n = 20

#initial disc configuration
#findpeak tries to find an equidistant configuration with a peak at f
initdist = findpeak(22.025e9,n)

#generate frequencies for calculation and for plotting
freqs = genFreqs(22.025e9,50e6; length=10) #optimize on these frequencies
freqsplot = genFreqs(22.025e9,150e6; length=1000)

#initialize physical properties of the booster
booster = AnalyticalBooster(initdist)
#=
Booster(
    pos,              disc positions
    ndisk,            disc number
    thickness,        disc thickness
    epsilon,          disc epsilon
    vmotor,           motor speed
    maxlength,        maximum allowed booster length (not implemented yet)
    timestamp,        booster operation time
    summedtraveltime, summed motor operation time
    codetimestamp     code runtime
)
=#

#initialize storage of measurements
hist = initHist(booster,10000,freqs,ObjAnalytical)
#                           ^ length, should be at least n
;

Objective Value: -243.3, Timestamp: 2023-06-05T13:10:43.589


### Optimizer

In [ ]:
#=
how to use

neldermead(                     directly modifies booster.pos
    booster,                    
    hist,                       
    freqs,                      frequency range to optimize on
    α,                          reflection parameter, 1 recommended
    β,                          expansion parameter, 1+2/n recommended
    γ,                          contraction parameter, 0.75-1/2n recommended
    δ,                          shrink parameter, 1-1/n recommended
    (objective,[options]),      objective function
    (initsimplex,[options]),    constructs initial simplex
    (simplexobj,[options]),     sweeps all vertices for their objective value
    (unstuckinator,[options]);  tries to unstuck system from bad local minima (not yet implemented)
    ϵgrad=0,                    terminate if norm(gradient) falls below this value,
    maxiter=100,                maximum allowed iterations,
    showtrace=false,            wether to print progress,
    showevery=1,                only print every i iterations,
    unstuckisiter=true,         wether an unstucking iteration counts towards maxiter
                                    warning! can run endlessly if false
    tracecentroid=false,        wether to write the centroid to trace, increases operation time
    traceevery=1,               write only every ith iteration to trace
)

returns trace, access with
trace[iteration].x      simplex disc positions
                .obj    simplex objective values
                .x_     centroid disc position if traced
                .obj_   centroid objective value if traced
                .t      timestamp
                .T      summed travel time
=#

In [3]:
trace = nelderMead(booster,hist,freqs,
                    1.,1+2/n,0.75-1/2n,1-1/n,
                    ObjAnalytical,
                    InitSimplexCoord(1e-4),
                    DefaultSimplexSampler,
                    UnstuckDont;
                    maxiter=Int(1e3),
                    showtrace=true,
                    showevery=100,
                    unstuckisiter=true);

Iter: 100, timestamp: 

105577 weeks, 2 days, 13 hours, 13 minutes, 7 seconds
Iter finished. Objective value: -1417.108

Iter: 200, timestamp: 105577 weeks, 2 days, 13 hours, 14 minutes, 12 seconds
Iter finished. Objective value: -4613.62

Iter: 300, timestamp: 105577 weeks, 2 days, 13 hours, 15 minutes, 15 seconds
Iter finished. Objective value: -12760.172

Iter: 400, timestamp: 105577 weeks, 2 days, 13 hours, 15 minutes, 54 seconds
Iter finished. Objective value: -13733.936



Iter: 500, timestamp: 105577 weeks, 2 days, 13 hours, 16 minutes, 17 seconds
Iter finished. Objective value: -13867.682

Iter: 600, timestamp: 105577 weeks, 2 days, 13 hours, 16 minutes, 27 seconds
Iter finished. Objective value: -14088.231

Iter: 700, timestamp: 105577 weeks, 2 days, 13 hours, 16 minutes, 30 seconds
Iter finished. Objective value: -14135.984

Iter: 800, timestamp: 105577 weeks, 2 days, 13 hours, 16 minutes, 32 seconds
Iter finished. Objective value: -14161.093



Iter: 900, timestamp: 105577 weeks, 2 days, 13 hours, 16 minutes, 33 seconds
Iter finished. Objective value: -14186.297

Iter: 1000, timestamp: 105577 weeks, 2 days, 13 hours, 16 minutes, 34 seconds
Iter finished. Objective value: -14211.565



MethodError: MethodError: no method matching *(::Dates.DateTime, ::Bool)

Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:578
  *(!Matched::StridedArray{P}, ::Real) where P<:Dates.Period
   @ Dates C:\Users\bergermann\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\Dates\src\deprecated.jl:44
  *(!Matched::Union{SparseArrays.AbstractCompressedVector{Tv, Ti}, SubArray{Tv, 1, <:SparseArrays.AbstractSparseMatrixCSC{Tv, Ti}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, false}, SubArray{Tv, 1, <:SparseArrays.AbstractSparseVector{Tv, Ti}, Tuple{Base.Slice{Base.OneTo{Int64}}}, false}} where {Tv, Ti}, ::Number)
   @ SparseArrays C:\Users\bergermann\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\SparseArrays\src\sparsevector.jl:1650
  ...


In [ ]:
#obtain result, booster is now in optimized position
#pos2dist(booster.pos)     #in distance space
booster.pos               #in position space

In [ ]:
#get handy output
plt = analyse(hist,trace,freqsplot; freqs=freqs,div=10)

#savefig(plt[i],"cool_epic_result_wow.svg")